In [12]:
!pip install Ollama -q
!pip install langchain -q
!pip install langchain_community -q

In [13]:
import ollama
from langchain_community.llms import Ollama


In [15]:
llm = Ollama(model="llama3", temperature = 0)

In [42]:
promptInicial = f"""
Dado el siguiente artículo, genera preguntas y respuestas relacionadas con el contenido del texto.

El formato debe ser:

Pregunta: <pregunta>
Respuesta: <respuesta>
Pregunta: <pregunta>
Respuesta: <respuesta>
... (y así sucesivamente)

La salida solo debe contener este formato, ni saludos, ni aclaraciones, solamente lo que se indica.

Artículo:
"""


In [66]:
promptInicial2 = f"""
Given the following article, generate deep questions and answers related to the content of the text. The questions should explore the details and nuances of the article, without referencing any other question or the article's numbering. The format should be:

Pregunta: <pregunta>
Respuesta: <respuesta>
Pregunta: <pregunta>
Respuesta: <respuesta>
... (and so on)

The output should only contain this format, no greetings, no explanations, only what is indicated. What you generate must be in Spanish.


This is the article:
"""

In [53]:
articulo_ejemplo = f"""
Las convocatorias para la designación por Concurso de Profesores Ordinarios titulares, asociados y adjuntos para las diferentes cátedras, áreas, núcleos, asignaturas que integran los núcleos, disciplinas, laboratorio de las Facultades que integran esta Universidad se regirán por las disposiciones del presente Reglamento y las que en consecuencia dicten las Unidades Académicas, que deberán ser aprobadas por el Consejo Superior.
"""


In [39]:
def generate_qa(article, prompt):
    formatted_prompt = f" {prompt} {article}"
    response = ollama.chat(model = 'llama3',
                           messages = [{'role':'user', 'content': formatted_prompt}],
                           options = {'temperature':0})
    return response['message']['content']

In [58]:
ejemplo = generate_qa(articulo_ejemplo, promptInicial2)

In [59]:
print(ejemplo)

Pregunta: ¿Qué tipo de convocatorias se realizarán para la designación de profesores?
Respuesta: Convocatorias para la designación por Concurso de Profesores Ordinarios titulares, asociados y adjuntos.

Pregunta: ¿Qué áreas o disciplinas serán objeto de estas convocatorias?
Respuesta: Las diferentes cátedras, áreas, núcleos, asignaturas que integran los núcleos, disciplinas y laboratorios de las Facultades que integran esta Universidad.

Pregunta: ¿Quién será responsable de dictar disposiciones para la designación de profesores?
Respuesta: Las Unidades Académicas, que deberán ser aprobadas por el Consejo Superior.

Pregunta: ¿Qué papel jugará el Consejo Superior en este proceso?
Respuesta: Aprobar las disposiciones que dicten las Unidades Académicas.


In [67]:
##Obsevaciones:

 ## Parece que los prompts laburan mejor en inglés

 ## Si le quito el documento/articulo/capitulo parece que las pregunta y respuestas no hacen TANTA referencia al documento/convocatoria/capitulo

 ## Parece que se puede toquear el prompt hasta que dé mejores resultado, de todas formas con un modelo avanzado andaría mejor? 

 ## De todas formas parece que el fine tuning no influye muco en que un modelo adquiera nuevo conocimiento, sino en poder modificar el comportamiento 
    #en la respuesta de los modelos, para lo otro sería mas útil usar un RAG
    #Dicho esto, ¿Vale la pena que el data set tenga preguntas totalmente exactas y profundas si parece que no puede adquirir conocimiento?

 ## Lo que si se podría hacer con el fine tuning capaz es darle la capacidad para que tenga un formato de respuesta que responda con el
    # DOCUMENTO - ARTICULO - CAPITULO; Capaz también puede adquirir mejor entendimiento de las palabras técnicas que tiene el documento;  
    